In [10]:
##### Imports ######
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import json

In [39]:
# Removes XML tags from the data
def clean_data(data):
    try:
        tags = ET.fromstring(str(data))
        return ET.tostring(tags, method='text').decode("utf-8")
    except:
        return data

# Finds all different places and removes the XML tags from them
def clean_places(places):
    placelist = []
    for i in places:
        tags = ET.fromstring(str(i))
        placelist.append(ET.tostring(tags, method='text').decode("utf-8"))
        
    return placelist

# Removes title, place and the XML tags from the text
def clean_text(text):
    for tag in text.findAll():
        tag.replaceWith('')

    return text.text.replace('\n','')

# Creates dictionary with all documents in the Reuters database, assigns doc_ID to all documents. 
def load_data():
    data_dict = {}
    data_id = 0
    data_str = ''
    for i in range(8):
        data1 = open('reuters_data/reut2-00'+ str(i) +'.sgm')
        soup = BeautifulSoup(data1,'lxml')
        items = soup.findAll('reuters')
        
        for doc in items:
            ID =     str({'docID':data_id})
            data =   str({'date':clean_data(doc.date),
                          'topic':clean_data(doc.topics), 
                          'place':clean_places(doc.places.findAll('d')), 
                          'people':clean_data(doc.people), 
                          'orgs':clean_data(doc.orgs), 
                          'exchanges':clean_data(doc.exchanges), 
                          'companies':clean_data(doc.companies), 
                          'title':clean_data(doc.title), 
                          'text':clean_text(doc.findAll('text')[0]) 
                      } )
            data_str = data_str + '\n' + ID + '\n' + data
            print('Document', data_id, 'of 8000', end='\r')
            data_id += 1
    
    with open('reuters.json', 'w') as f:
        json.dump(data_str, f)
    
    return 'done'

load_data()

Document 7999 of 8000369 of 8000 569 of 8000 745 of 8000886 of 8000 1141 of 80001280 of 8000 1418 of 8000 1542 of 8000 1657 of 8000 1773 of 8000 1902 of 8000 2140 of 8000 of 8000 2593 of 8000 2713 of 8000 2816 of 8000 2924 of 8000 3118 of 8000 3229 of 80003327 of 8000 of 8000 3577 of 8000 3693 of 8000 3800 of 8000 3916 of 8000 4154 of 8000 4406 of 8000 4522 of 8000 4628 of 8000 4735 of 8000 4873 of 8000 5115 of 8000 5348 of 8000 5479 of 8000 of 8000 5679 of 8000 5873 of 8000 5960 of 8000 6103 of 8000 6203 of 80006297 of 8000 6398 of 8000 6485 of 8000 6664 of 8000 of 8000 6839 of 8000 6918 of 8000 7097 of 80007195 of 80007292 of 8000 7387 of 8000 7480 of 8000 7663 of 8000 7755 of 8000 7847 of 8000 7926 of 8000
{'docID': 0}
{'text': 'Showers continued throughout the week inthe Bahia cocoa zone, alleviating th


'done'

In [ ]:
with open(DATA_FILENAME, mode='w', encoding='utf-8') as feedsjson:
    entry = {'name': args.name, 'url': args.url}
    feeds.append(entry)
    json.dump(feeds, feedsjson)